In [27]:
from nets import Critic, Actor
import numpy as np

In [28]:
critic = Critic(nature="primary", dolinar_layers=1)
context = np.reshape(np.array([critic.pad_value, 0.]), (1,1,2))
critic(context)
critic.trainable_variables


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



[<tf.Variable 'critic_5/lstm_5/lstm_cell_5/kernel:0' shape=(2, 2000) dtype=float32, numpy=
 array([[ 0.02683562, -0.00300943, -0.00117685, ..., -0.03192975,
          0.01224592,  0.04255226],
        [-0.04850321,  0.05066887,  0.04943989, ..., -0.05051254,
          0.02794592,  0.00341985]], dtype=float32)>,
 <tf.Variable 'critic_5/lstm_5/lstm_cell_5/recurrent_kernel:0' shape=(500, 2000) dtype=float32, numpy=
 array([[-0.01926458, -0.0216778 , -0.01512166, ...,  0.00278797,
          0.01074278,  0.03477448],
        [-0.00655923,  0.01254892, -0.00288169, ...,  0.03123787,
          0.02637848,  0.00847593],
        [-0.00167228, -0.02720782, -0.01590514, ..., -0.02778661,
          0.02031054, -0.02542397],
        ...,
        [ 0.00858968, -0.02798094, -0.03235864, ..., -0.02285349,
          0.02787674, -0.01697049],
        [-0.03451683,  0.00556748, -0.0079088 , ..., -0.00476526,
         -0.00467099,  0.01305982],
        [ 0.00789589,  0.02097274,  0.0017405 , ...,  0.00506

In [29]:
critic.load_weights("results/run_14/networks/critic_primary/")

In [30]:
critic.trainable_variables

[<tf.Variable 'critic_5/lstm_5/lstm_cell_5/kernel:0' shape=(2, 2000) dtype=float32, numpy=
 array([[ 0.1344623 ,  0.10571895,  0.02195035, ...,  0.03437563,
         -0.06395639,  0.0417714 ],
        [-0.19678222,  0.13549855,  0.07718753, ..., -0.02196645,
         -0.10053977,  0.17262664]], dtype=float32)>,
 <tf.Variable 'critic_5/lstm_5/lstm_cell_5/recurrent_kernel:0' shape=(500, 2000) dtype=float32, numpy=
 array([[-0.01661026,  0.02466727, -0.01617306, ...,  0.002399  ,
         -0.02548583, -0.01318562],
        [-0.02239862, -0.02083812, -0.01693802, ...,  0.00941509,
          0.01611612, -0.02235344],
        [-0.01548921,  0.01572568,  0.03005023, ...,  0.02207434,
          0.0062186 ,  0.00069975],
        ...,
        [ 0.00887478, -0.00958829,  0.00322336, ...,  0.01526765,
          0.01413793,  0.02274813],
        [ 0.02618111,  0.01596071,  0.03140911, ...,  0.0411947 ,
          0.07655381,  0.01259478],
        [-0.00732564, -0.00672578,  0.02698318, ...,  0.00431

In [43]:
actor = Actor(nature="target",dolinar_layers=1)
actor.load_weights("results/run_14/networks/actor_primary/")

*****

*****

In [44]:
from misc import PolicyEvaluator

In [113]:
class PolicyEvaluator(Basics):
    def __init__(self, **kwargs):
        amplitude= kwargs.get("amplitude", .4)
        dolinar_layers=kwargs.get("dolinar_layers", 2)
        number_phases=kwargs.get("number_phases", 2)
        super().__init__(amplitude=amplitude, dolinar_layers=dolinar_layers, number_phases=number_phases)

        displacement_tree = {}
        trajectory_tree = {}
        #self.at = make_attenuations(self.number_layers)
        for layer in range(self.dolinar_layers+1):
            displacement_tree[str(layer)] = {}
            trajectory_tree[str(layer)] = {}

        for k in outcomes_universe(self.dolinar_layers):
            for layer in range(self.dolinar_layers+1):
                displacement_tree[str(layer)][str(k[:layer])] = 0.
                trajectory_tree[str(layer)][str(k[:layer])] = []

        self.history_tree = displacement_tree
        self.recorded_trajectory_tree = trajectory_tree
        self.recorded_trajectory_tree_would_do = trajectory_tree.copy()

    def save_hisory_tree(self, directory):
        output = open(directory+"/history_tree.pkl", "wb")
        pickle.dump(self.recorded_trajectory_tree, output)
        output.close()
        output = open(directory+"/history_tree_would_do.pkl", "wb")
        pickle.dump(self.recorded_trajectory_tree_would_do, output)
        output.close()

    def load_history_tree(self, directory):
        opp = open(directory+"/history_tree.pkl", "rb")
        self.recorded_trajectory_tree_loaded = pickle.load(opp)
        opp = open(directory+"/history_tree_would_do.pkl", "rb")
        self.recorded_trajectory_tree_loaded_would_do = pickle.load(opp)
        print("recorded_trajectory_tree_loaded")
        return

    def random_tree(self):
        actions = self.history_tree.copy()
        for k in outcomes_universe(self.dolinar_layers):
            for layer in range(self.dolinar_layers+1):
                actions[str(layer)][str(k[:layer])] = np.random.random()
        return actions

    def success_probability(self, displacements_tree):
        """
        Given a tree of conditional actions (on the outcomes history), computes
        the success probability. Notice the final action is the guess
        for the phase of the state given a given branch.
        """
        p=0
        for ot in outcomes_universe(self.dolinar_layers):
            c=1
            for layer in range(self.dolinar_layers):
                eff_at = np.prod(np.sin(self.at[:layer]))*np.cos(self.at[layer])
                c*=P(displacements_tree[str(self.dolinar_layers)][str(ot)]*self.amplitude,
                     displacements_tree[str(layer)][str(ot[:(layer)])], eff_at, ot[self.dolinar_layers-1-layer] ) #notice i respect that the columns of the outcomes_universe
                #correspond to the layer: the last column is the first layer.
            p += c
        return p/self.number_phases

    
    def give_max_lik_guess(self, history):
        prob=np.ones(self.number_phases)
        for layer in range(int(len(history)/2)):
            effective_attenuation = np.prod(np.sin(self.at[:layer]))*np.cos(self.at[layer])#Warning one!
            prob*=np.array([self.P(phase_guess*self.amplitude, history[layer], effective_attenuation, history[layer+1]) for
                            phase_guess in self.possible_phases])
        return self.possible_phases[np.argmax(prob)]

    def greedy_strategy(self, actor, critic):
        """
        Assuming actor, critic and self have the same dolinar_layers.
            self.possible_phases are the possible phases of the coherent states

        """

        rr = np.ones((2**(self.dolinar_layers-1), self.dolinar_layers, 1))*actor.pad_value
        if self.dolinar_layers != 1:
            rr[:,1:] = np.reshape(outcomes_universe(self.dolinar_layers-1),(2**(self.dolinar_layers-1), self.dolinar_layers-1,1))

        actor.lstm.stateful = False
        preds = np.squeeze(actor(rr))
        actor.lstm.stateful = True

        if self.dolinar_layers==1:
            history = preds
            self.history_tree[str(0)][str([])] = preds
            for final_outcome in [[0], [1]]:
                final_history = np.append(history, final_outcome)
                #print(final_history)
                #print(final_outcome)
                #print(self.give_max_lik_guess(final_history))
                #self.history_tree[str(self.dolinar_layers)][str(final_outcome)] = self.possible_phases[critic.give_favourite_guess(final_history)[0]]
                self.history_tree[str(self.dolinar_layers)][str(final_outcome)] = self.give_max_lik_guess(final_history)
                
            return self.success_probability(self.history_tree)
        else:

            for ot, seqot in zip(outcomes_universe(self.dolinar_layers-1), preds):
                for layer in range(self.dolinar_layers):
                    self.history_tree[str(layer)][str(ot[:layer])] = seqot[layer]

                history = []
                index_seqot, index_ot= 0, 0
                for index_history in range(2*self.dolinar_layers-1):
                    if index_history%2==0:
                        history.append(seqot[index_seqot])
                        index_seqot+=1
                    else:
                        history.append(ot[index_ot])
                        index_ot+=1
                for final_outcome in [[0],[1]]:
                    final_history = np.append(history, final_outcome)
                    #self.history_tree[str(self.dolinar_layers)][str(np.append(ot,final_outcome))] = self.possible_phases[critic.give_favourite_guess(final_history)[0]]
                    self.history_tree[str(self.dolinar_layers)][str(np.append(ot,final_outcome))] = self.give_max_lik_guess(final_history)
                    

        return self.success_probability(self.history_tree)


In [117]:
pol = PolicyEvaluator(dolinar_layers = 2)

In [118]:
pol.greedy_strategy(actor=actor, critic=critic)

0.624651092947005

In [116]:
critic

In [103]:
pol.history_tree

{'0': {'[]': array(-0.2537048, dtype=float32)},
 '1': {'[0]': 0.0, '[1]': 0.0, '0': (1+0j), '1': (-1+0j)}}